<a href="https://colab.research.google.com/github/rajagopal17/spacy-notebooks/blob/master/Glove_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', size=14)
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)
import csv
import re
from __future__ import unicode_literals
import spacy
from spacy.tokens import doc
nlp=spacy.load('en')
import en_core_web_sm
#nlp=en_core_web_md.load()
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
parser = English()
import string
punctuations=string.punctuation
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!git clone https://github.com/MohammadWasil/Sentiment-Analysis-IMDb-Movie-Review.git

Cloning into 'Sentiment-Analysis-IMDb-Movie-Review'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70
Unpacking objects: 100% (70/70), done.


In [3]:
data_train =pd.read_csv('/content/Sentiment-Analysis-IMDb-Movie-Review/labeledTrainData.tsv', delimiter='\t',encoding="utf-8")
#data_test =('/content/Sentiment-Analysis-IMDb-Movie-Review/testData.tsv')
data_file = data_train[['sentiment','review']].copy()
train_ds = data_file.head(15000).copy()
test_ds  = data_file.tail(5000).copy()
train_ds.to_csv('/content/train_ds.csv',index=False)
test_ds.to_csv('/content/test_ds.csv',index=False)
train_ds.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


# Convert column to string, apply nlp pipe lines for lemmatization,ents, etc
  

1.   Convert column to list and feed it back to new data frame
2.   Get frequency and plot on graph



In [ ]:
#Convert each row of reviews column to string and store it in a file

temp_file=data_train['Reviews'].apply(lambda x:str(x))
temp_file=data_train['Reviews'].apply(lambda x:re.sub('[^A-Z 0-9 a-z-]+','',x))
temp_file

In [ ]:
lemma_list=list([token.lemma_ for token in doc if token.is_stop==False] for doc in nlp.pipe(temp_file, n_threads=2,batch_size=1000,disable=['tagger','parser','ner']))


In [ ]:
lemma_names=list([token.text for token in doc if token.pos_=='PROPN'] for doc in nlp.pipe(temp_file,batch_size=1000))

In [ ]:
lemma_person=list([token.text for token in doc if token.ent_type_=='PERSON'] for doc in nlp.pipe(temp_file,batch_size=1000))

In [ ]:
get_names=[]
for x in lemma_person:
  for y in x:
    get_names.append(y)
get_names

In [ ]:
data_clean['lemma']=lemma_list
data_clean['Reviews']=lemma_names
data_clean['names']=lemma_person

In [ ]:
data_clean.head(20)

In [ ]:
from collections import Counter
word_freq=Counter(get_names)
word_freq



In [ ]:
word_freq_graph =pd.DataFrame(list(word_freq.items()),columns=['name','freq'])
word_freq_graph
final_df=word_freq_graph[word_freq_graph['freq']>800 ]
final_df

In [ ]:
import matplotlib.pyplot as plot
final_df.plot.barh(x='name', y='freq', title="Frequency of the mention of lead character's name");

plot.show(block=True);

# Torch Text

In [ ]:
from torchtext.data import Field,TabularDataset,BucketIterator

In [ ]:
#X_train,X_test,y_train,y_test = train_test_split('/content/Sentiment-Analysis-IMDb-Movie-Review/labeledTrainData.tsv')

In [ ]:
!git clone https://github.com/AladdinPerzon/Machine-Learning-Collection.git

Cloning into 'Machine-Learning-Collection'...
remote: Enumerating objects: 9466, done.
remote: Total 9466 (delta 0), reused 0 (delta 0), pack-reused 9466
Receiving objects: 100% (9466/9466), 1.07 GiB | 39.99 MiB/s, done.
Resolving deltas: 100% (661/661), done.
Checking out files: 100% (154/154), done.


In [ ]:
def tokenize(text):
  return[token.text for token in nlp(text)]

In [ ]:
#tokenize= lambda x: x.split()
TEXT  = Field(sequential= True,use_vocab=True,tokenize=tokenize,lower=True)
LABEL = Field(sequential=False,use_vocab=False)
fields={'sentiment':LABEL,'review':TEXT}

#Split dataset for train & test

In [ ]:
#train_data, test_data= TabularDataset(path='/content/sample_data/test_d.csv',format='csv',fields={'sentiment':sentiment,'review':review})
train = TabularDataset(path='/content/sample_data/train_ds.csv', 
                        format='csv', 
                        fields=[("sentiment",LABEL),
                                ("review",TEXT)],  
                        skip_header=True)
test = TabularDataset(path='/content/sample_data/test_ds.csv', 
                        format='csv', 
                        fields= [("sentiment",LABEL),
                                ("review",TEXT)],  
                        skip_header=True)                             
                       

In [ ]:
TEXT.build_vocab(train,max_size=10000,min_freq=2)

In [ ]:
TEXT.vocab.itos

#TEXT.vocab.stoi['movie']

['<unk>',
 '<pad>',
 'the',
 ',',
 '.',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'that',
 'this',
 "'s",
 '-',
 'was',
 '/><br',
 'as',
 'with',
 '"',
 'for',
 'but',
 'movie',
 'film',
 ')',
 'on',
 '(',
 'you',
 'are',
 "n't",
 'not',
 'his',
 'he',
 'have',
 'be',
 'one',
 'all',
 'at',
 'by',
 'an',
 'who',
 'they',
 'from',
 'like',
 'her',
 '!',
 'so',
 "'",
 'about',
 'there',
 'or',
 'has',
 'just',
 'out',
 'good',
 'some',
 'do',
 'if',
 'what',
 'she',
 'more',
 'very',
 'when',
 '?',
 'up',
 'would',
 'no',
 'even',
 'their',
 'time',
 'my',
 'can',
 'which',
 'only',
 'had',
 'really',
 'were',
 'story',
 'did',
 'see',
 'people',
 ':',
 'me',
 'does',
 'than',
 'we',
 'could',
 'well',
 'been',
 'will',
 'get',
 'into',
 'most',
 'much',
 'great',
 'other',
 'because',
 'him',
 'how',
 'first',
 '...',
 'bad',
 'also',
 'then',
 'them',
 'its',
 'made',
 'make',
 'any',
 'way',
 'after',
 'too',
 '/>the',
 'movies',
 'think',
 'characters',
 '<',
 'br',
 'f

In [ ]:
train_ds.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


# Glove Embeddings

In [4]:
X=train_ds[['review']].copy()
y=train_ds[['sentiment']].copy()

Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,train_size=0.7,random_state=32,shuffle=True)

In [ ]:
lines=Xtest['review'].apply(lambda x: str(x))

lines_list=list([token.lemma_.lower() for token in doc if token.is_alpha and token.is_stop==False] for doc in nlp.pipe(lines, batch_size=1000,disable=['tagger','parser','ner']))

lines_list

# How to use standard Glove vectors for finding similarities without Gensim

To load the pre-trained vectors, we must first create a dictionary that will hold the mappings between words, and the embedding vectors of those words.

In [ ]:
import os
import numpy as np
GLOVE_DIR ='/content/drive/My Drive/Python'
print('Indexing word vectors.')
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'),encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

#print(embeddings_index['banana'])

In [79]:
from scipy import spatial
def find_closest_embeddings(embedding):
    return sorted(embeddings_index.keys(), key=lambda word: spatial.distance.euclidean(embeddings_index[word], embedding))


In [80]:
find_closest_embeddings(embeddings_index["japanese"])[:10]

['japanese',
 'japan',
 'tokyo',
 'korean',
 'chinese',
 'taiwanese',
 'kyodo',
 'hashimoto',
 'moreover',
 'meanwhile']

# Converting  Standard Glove vectors into Gensim- Word2Vec format for finding similarities

In [10]:
!pip3 install glove_python
#https://medium.com/analytics-vidhya/word-vectorization-using-glove-76919685ee0b
#https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db

     |████████████████████████████████| 266kB 2.7MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700256 sha256=59040056b7f2464391c77d5e211f15d479ea812527654a2ccc6c39b6000a923f
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [67]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file='/content/drive/My Drive/Python/glove.6B.300d.txt', word2vec_output_file="/content/drive/My Drive/Python/gensim_glove_vectors.txt")    
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format("/content/drive/My Drive/Python/gensim_glove_vectors.txt", binary=False)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [74]:

model.most_similar('japanese')



/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('japan', 0.769140362739563),
 ('tokyo', 0.6243131160736084),
 ('korean', 0.6134214401245117),
 ('chinese', 0.5663821697235107),
 ('yen', 0.4867936968803406),
 ('taiwanese', 0.4863933324813843),
 ('hashimoto', 0.46149420738220215),
 ('asian', 0.45883551239967346),
 ('kyodo', 0.45841461420059204),
 ('foreign', 0.4445464611053467)]

# Train Glove model on own corpus, find similarities and save the model

https://github.com/alexeygrigorev/avito-duplicates-kaggle/blob/master/prepare_glove_model.py

In [50]:
from time import time

def train_glove(sentences):
    print ('training glove model...')
    t0 = time()
    
    num_features = 300    # Word vector dimensionality
    context = 5          # Context window size
    learning_rate = 0.05
    
    corpus = Corpus()
    corpus.fit(sentences, window=context)

    glove = Glove(no_components=num_features, learning_rate=learning_rate)
    glove.fit(corpus.matrix, epochs=30, no_threads=8, verbose=True)
    glove.add_dictionary(corpus.dictionary)

    print('took %0.5fs.' % (time() - t0))
    return (glove)

In [54]:
gl_model=train_glove(lines_list)

training glove model...
Performing 30 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
took 171.18543s.


In [85]:
gl_model.most_similar('cast',number=6)

[('support', 0.9472058752096072),
 ('crow', 0.9320960528668345),
 ('perfectly', 0.9217136045605772),
 ('ensemble', 0.9155379812714399),
 ('rest', 0.9066026077791719)]

In [47]:
gl_model.save('glove_final')

In [48]:
rmodel=gl_model.load('glove_final')

In [86]:
rmodel.most_similar('cast',number=6)

[('supporting', 0.8433521223781817),
 ('excellent', 0.8331830963459859),
 ('fine', 0.809570443868265),
 ('perfectly', 0.7971040169644816),
 ('great', 0.7901401985065863)]